##### Copyright 2021 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
import os
import numpy as np
from scipy.io import wavfile
from scipy.signal import resample
import tensorflow.compat.v1 as tf
from tensorflow.io import gfile
from IPython.display import Audio, display
from ipywidgets import widgets

In [ ]:
# @title Mount Google Drive
from google.colab import drive
ROOT_DIR = '/content/gdrive'
drive.mount(ROOT_DIR, force_remount=True)

In [ ]:
# @title Helper function for playing audio.
def PlaySound(samples, sample_rate=16000):
  out = widgets.Output()
  with out:
    display(Audio(samples, rate=sample_rate))
  display(out)

In [ ]:
# @title Helper class for separation model inference.
class SeparationModel(object):
  """Tensorflow audio separation model."""

  def __init__(self,
               checkpoint_path,
               metagraph_path):
    self.graph = tf.Graph()
    self.sess = tf.Session(graph=self.graph)
    with self.graph.as_default():
      new_saver = tf.train.import_meta_graph(metagraph_path)
      new_saver.restore(self.sess, checkpoint_path)
    self.input_placeholder = self.graph.get_tensor_by_name(
        'input_audio/receiver_audio:0')
    self.output_tensor = self.graph.get_tensor_by_name('denoised_waveforms:0')


  def separate(self, mixture_waveform):
    """Separates a mixture waveform into sources.

    Args:
      mixture_waveform: numpy.ndarray of shape (num_mics, num_samples).

    Returns:
      numpy.ndarray of separated waveforms of shape (num_sources, num_samples).
      dict of additional tensor outputs.
    """
    mixture_waveform_input = np.expand_dims(mixture_waveform, 0)
    feed_dict = {self.input_placeholder: mixture_waveform_input}

    separated_waveforms = self.sess.run(self.output_tensor, feed_dict=feed_dict)
    return separated_waveforms[0]

In [ ]:
# @title Download the pre-trained speech enhancement model files.
%%shell
gsutil cp -r gs://gresearch/cochlear_implant/speech_enhancement_model /tmp/

In [ ]:
# @title Load speech enhancement model.
MODEL_PATH = '/tmp/speech_enhancement_model'

checkpoint = os.path.join(MODEL_PATH, 'checkpoint')
metagraph = os.path.join(MODEL_PATH, 'inference.meta')
model = SeparationModel(checkpoint, metagraph)

In [ ]:
# @title Get some wav paths.
PATH_AUDIO = 'gdrive/My Drive/cihack_audio'  # E.g. /content/gdrive/My Drive/cihack_audio
PATH_ENHANCED = PATH_AUDIO + '_enhanced'

audio_clip_matcher = '*.wav'  #@param
wavs = gfile.glob(os.path.join(PATH_AUDIO, audio_clip_matcher))

In [ ]:
#@title File read/write functions.
def write_wav(filename, waveform, sample_rate=16000):
  """Write a audio waveform (float numpy array) as .wav file."""
  wavfile.write(
      filename, sample_rate,
      np.round(np.clip(waveform * 2**15, -32768, 32767)).astype(np.int16))

def read_wav(wav_path, sample_rate=16000, channel=None):
  """Read a wav file as numpy array.

  Args:
    wav_path: String, path to .wav file.
    sample_rate: Int, sample rate for audio to be converted to.
    channel: Int, option to select a particular channel for stereo audio.

  Returns:
    Audio as float numpy array.
  """
  sr_read, x = wavfile.read(wav_path)
  x = x.astype(np.float32) / (2**15)

  if sr_read != sample_rate:
    x = resample(x, int(round((float(sample_rate) / sr_read) * len(x))))
  if x.ndim > 1 and channel is not None:
    return x[:, channel]
  return x

In [ ]:
# @title Enhance some wavs and play audio.
for wav in wavs:
  print('wav path:', wav)
  print('Input')
  receiver_audio = read_wav(wav)

  PlaySound(receiver_audio, 16000)

  enhanced = model.separate(receiver_audio[np.newaxis])
  output_path = os.path.join(PATH_ENHANCED, os.path.basename(wav))
  output_path = os.path.join(PATH_ENHANCED, os.path.basename(wav))
  gfile.makedirs(os.path.dirname(output_path))
  write_wav(output_path, enhanced.T, 16000)

  print('Speech estimate')
  PlaySound(enhanced[0], 16000)
  print('Noise estimate')
  PlaySound(enhanced[1], 16000)